In [1]:
import json, numpy as np, pretty_midi as pm
from IPython.display import Audio, display

VOCAB_PATH = "../data/processed/dataset_2/vocab_topN.json"
SAMPLES_JSONL = "../outputs/samples_bars.jsonl"
SF2_PATH = r"C:\Soundfonts\FluidR3_GM.sf2"  # <-- ajusta a tu ruta


ModuleNotFoundError: No module named 'pretty_midi'

In [ ]:
def load_vocab(vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
        voc = json.load(f)

    if isinstance(voc, dict) and "bitmask2id" in voc:
        id2bitmask = {int(k): str(v) for k, v in voc["id2bitmask"].items()}
        unk_id = voc.get("unk_id", None)
        return id2bitmask, unk_id

    # legado
    b2i = {str(k): int(v) for k, v in voc.items()}
    id2bitmask = {v: k for k, v in b2i.items()}
    return id2bitmask, b2i.get("UNK", None)

def read_jsonl_bars(jsonl_path):
    bars = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            bars.append(obj["tokens"])  # lista de 16 ids
    return bars

id2bitmask, unk_id = load_vocab(VOCAB_PATH)
bars = read_jsonl_bars(SAMPLES_JSONL)
len(bars), bars[0][:5]


In [ ]:
CLASS_PITCHES = [36, 38, 42, 46, 41, 45, 48, 49, 51]  # KICK,SNARE,HHc,HHo,TOML,TOMM,TOMH,CRASH,RIDE

def bars_to_pretty_midi(bars_ids, id2bitmask, bpm=100, note_len=0.06, velocity=95):
    midi = pm.PrettyMIDI(initial_tempo=bpm)
    drum = pm.Instrument(program=0, is_drum=True, name="Drums")

    spb = 60.0 / float(bpm)
    bar_dur = 4.0 * spb
    step = bar_dur / 16.0

    t0 = 0.0
    for bar in bars_ids:
        # ids -> bitmask strings
        bitmasks = []
        for tid in bar:
            bm = id2bitmask.get(tid, "000000000")
            bm = bm if isinstance(bm, str) else format(int(bm), "09b")
            bm = bm.zfill(9)
            bitmasks.append(bm)

        for i, bm in enumerate(bitmasks):
            for cls_idx, ch in enumerate(bm):
                if ch == "1":
                    pitch = CLASS_PITCHES[cls_idx]
                    start = t0 + i * step
                    end = start + note_len
                    drum.notes.append(pm.Note(velocity=velocity, pitch=pitch, start=start, end=end))
        t0 += bar_dur

    midi.instruments.append(drum)
    return midi

midi_obj = bars_to_pretty_midi(bars[:8], id2bitmask, bpm=100)  # reproduce 8 compases


In [ ]:
# Requiere pyfluidsynth instalado y SF2_PATH válido
audio = midi_obj.fluidsynth(sf2_path=SF2_PATH, sample_rate=44100)
display(Audio(audio, rate=44100))


In [ ]:
midi_obj.write("../outputs/notebook_preview.mid")
print("MIDI guardado en outputs/notebook_preview.mid")
